# Exploration Notebook to scrape several websites for job openings, store them in a database and build an own recommender for interesting jobs

In [1]:
from bs4 import BeautifulSoup
import undercover_request # custom module, install: pip install git+https://github.com/TimSchopf/undercover_request

In [2]:
bosch_params = {'searchTerm':'thesis'}
daimler_params = {_: 1555346176442}

In [3]:
# Bosch requires a get request and blocks post requests
bosch_req = undercover_request.request('https://www.bosch.de/karriere/jobs/', request_type='get', params=bosch_params)

update user agent database
get proxy list
try proxies
SUCCESS Proxy: {'ip': '37.200.121.50', 'port': '38775'} User Agent: Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36


In [4]:
# Daimler uses an API to send the data to the client.
# The Daimler Carrer Website itself is just a HTML template without data.
# Data and template render client side and are sent seperately to the client.
# In this case we must find the API URL that returns the raw data (the Daimler API sends JSON).
daimler_api_url = 'https://global-jobboard-api.daimler.com/v3/search/%7B%22LanguageCode%22%3A%22EN%22%2C%22ScoreThreshold%22%3A80%2C%22SearchParameters%22%3A%7B%22MatchedObjectDescriptor%22%3A%5B%22PositionID%22%2C%22PositionTitle%22%2C%22PositionURI%22%2C%22LogoURI%22%2C%22OrganizationName%22%2C%22Organization%22%2C%22Organization.MemberCode%22%2C%22ParentOrganization%22%2C%22PositionLocation.CityName%22%2C%22PositionLocation.Longitude%22%2C%22PositionLocation.Latitude%22%2C%22PositionIndustry.Name%22%2C%22JobCategory.Name%22%2C%22JobCategory.Code%22%2C%22CareerLevel.Name%22%2C%22CareerLevel.Code%22%2C%22Facet%3AParentOrganization%22%2C%22Facet%3APositionLocation.CityName%22%2C%22Facet%3APositionLocation.CountryName%22%2C%22PublicationStartDate%22%2C%22ParentOrganizationGenesisID%22%5D%2C%22FirstItem%22%3A0%2C%22CountItem%22%3A20%7D%2C%22SearchCriteria%22%3A%5B%7B%22CriterionName%22%3A%22PositionFormattedDescription.Content%22%2C%22CriterionValue%22%3A%5B%22data%22%5D%7D%2C%7B%22CriterionName%22%3A%22PublicationLanguage.Code%22%2C%22CriterionValue%22%3A%5B%22EN%22%5D%7D%2C%7B%22CriterionName%22%3A%22PublicationChannel.Code%22%2C%22CriterionValue%22%3A%5B%2212%22%5D%7D%2C%7B%22CriterionName%22%3A%22CareerLevel.Code%22%2C%22CriterionValue%22%3A%5B%2240%22%5D%7D%5D%7D?_=1555346176443'
daimler_req = undercover_request.request(daimler_api_url, request_type='post', params=daimler_params)

update user agent database
get proxy list
try proxies
ERROR Proxy: {'ip': '181.129.181.250', 'port': '53539'} User Agent: Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.67 Safari/537.36
ERROR Proxy: {'ip': '134.209.56.28', 'port': '3128'} User Agent: Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36
ERROR Proxy: {'ip': '186.148.168.91', 'port': '49186'} User Agent: Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; WOW64; Trident/4.0; SLCC2; Media Center PC 6.0; InfoPath.2; MS-RTC LM 8
ERROR Proxy: {'ip': '186.159.1.217', 'port': '56875'} User Agent: Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.2117.157 Safari/537.36
ERROR Proxy: {'ip': '194.150.143.7', 'port': '50691'} User Agent: Mozilla/5.0 (X11; NetBSD) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.116 Safari/537.36
ERROR Proxy: {'ip': '219.93.16.182', 'port': '36064'} User Agent: Mozil

In [5]:
daimler_data = daimler_req.json()
bosch_data = BeautifulSoup(bosch_req.text, 'html.parser')